# Prikupljanje podataka

## [**Web scraping**](https://realpython.com/beautiful-soup-web-scraper-python/) 

Web scraping je proces (automatiziranog) prikupljanja podataka s interneta. Prilikom web scrapinga bitno se informirati o mogućim [ograničenjima i zabranama](https://benbernardblog.com/web-scraping-and-crawling-are-perfectly-legal-right/). Benoit Bernard navodi neke od ključnih koraka kako bismo se osigurali od kršenja prava i odredbi vezanih uz podatke koje prikupljamo:

1. Koristite API ako je dostupan
2. Poštujte uvjete pružanja usluge (*engl. Terms of Service* )
3. Poštujte [*robots.txt*](https://www.inf.uniri.hr/robots.txt)
4. Koristite razuman *crawl rate* - ne pretjerujte sa slanjem upita 
5. ...



> *"You write your code once, and it will get the information you want many times and from many pages."*

### Istraživanje web sjedišta

Kako bismo efektivno automatizirali proces prikupljanja podataka, potrebno se pobliže upoznati sa samom strukturom web sjedišta. Na ovim vježbama, zadatak nam je preuzeti sve izvedbene programe kolegija na [**Fakultetu informatike i digitalnih tehnologija**](https://www.inf.uniri.hr/). Kroz interakcije s web sjedištem pronađite stranicu na kojoj se nalaze informacije o izvedbenim programima.



#### Analiza URL-a

Pojedini URL-ovi otkrivaju nam korisne informacije o samom web sjedištu, npr. iz URL-a:     

https://www.inf.uniri.hr/nastava/izvedbeni-programi 

moguće je pretpostaviti URL na kojem se nalaze izvedbeni programi prijediplomskog studija:  

https://www.inf.uniri.hr/nastava/izvedbeni-programi/prijediplomski-studij. 

Slično, možemo zaključiti da se izvedbeni programi diplomskog studija nalaze na URL-u:      

https://www.inf.uniri.hr/nastava/izvedbeni-programi/diplomski-studij. 

Sa zaključcima ne treba brzati, iako struktura postoji, **najčešće** nije konzistenta.

![](./Images/404_kategorija_nije_pronadena.png)

#### Analiza sjedišta razvojnim alatima (Developer Tools)

Razvojni alati omogućuju nam direktno uređivanje stranica, dijagnosticiranje problema i upoznavanje sa strukturom.

![](./Images/Dev_tools.png)



### Dohvaćanje podataka sa web sjedišta

Pomoću biblioteke [requests](https://www.w3schools.com/python/module_requests.asp) dohvatimo stranicu s izvedbenim programima prijediplomskog studija: https://www.inf.uniri.hr/nastava/izvedbeni-programi/prijediplomski-studij

In [ ]:
import requests 

URL = "https://www.inf.uniri.hr/nastava/izvedbeni-programi/prijediplomski-studij"
page = requests.get(URL)

print(page.text)

Ispisivanjem sadržaja `text` atributa objekta `page` vidimo HTML sadržaj identičan onome u *inspector* prozoru razvojnih alata. Odlična vijest za nas, dio web sjedišta koji nas interesira je statičan! Probajmo dohvatiti interesantne informacije, na primjer, sve nazive kolegija te njihove nositelje.

#### Dohvaćanje podataka split + regex

> ZADATAK 1. Dohvaćanje podataka o nazivu kolegija i nositelju pomoću metode `split()`

In [ ]:
# Korištenjem metode split

retci_tablice = page.text.split("<tr>")
print(retci_tablice)

for redak in retci_tablice:
    try:
        print(redak.split("<td>")[2].replace("</td>", ""))
        print(redak.split("<td>")[3].replace("</td>", ""))
        print("-"*20)
    except IndexError:
        print("")

> ZADATAK 2. Dohvaćanje linkova metodom `split()`

In [ ]:
for redak in retci_tablice:
    try:
        print(redak.split("<td>")[4])
    except IndexError:
        print("")

Kod složenijih primjera `split()` metoda brzo postaje neučinkovita. Dohvatimo linkove pomoću regularnih izraza.

> ZADATAK 3. Dohvaćanje linkova korištenjem biblioteke [re](https://docs.python.org/3/library/re.html); primjer: 

`<a href="/images/nastava/izvedbeni/2023_2024/PDS/1/DINP_FIDIT_2023_2024_OPROG.pdf" target="_blank"><i class="fa fa-file-pdf-o"></i></a></td>`



Metode `re.search()`, `re.match()`, `re.fullmatch()`:
- [re.match()](https://docs.python.org/3/library/re.html#re.match) checks for a match only at the beginning of the string
- [re.search()](https://docs.python.org/3/library/re.html#re.search) checks for a match anywhere in the string (this is what Perl does by default)
- [re.fullmatch()](https://docs.python.org/3/library/re.html#re.fullmatch) checks for entire string to be a match

- [re.compile()](https://docs.python.org/3/library/re.html#re.compile) compiles regular expression


In [ ]:
import re

# Primjer na jednom stringu
string = "<a href=\"/images/nastava/izvedbeni/2023_2024/PDS/1/DINP_FIDIT_2023_2024_OPROG.pdf\" target=\"_blank\"><i class=\"fa fa-file-pdf-o\"></i></a></td>"
regex = re.compile(r"href=\"([\w\/-_\.]*)\"")
                     
print(regex)
print(re.search(regex, string))



In [ ]:
# Primjer nad svi retcima tablice
linkovi = []
for redak in retci_tablice:
    try:
        linkovi.append(re.search(regex, redak).group(1))
        print(re.search(regex, redak).group(1))
    except AttributeError:
        print("")

In [ ]:
print(linkovi)

#### Dohvaćanje podataka pomoću biblioteke BeautifulSoup

> ZADATAK 4. Dohvaćanje podataka o nazivu kolegija i nositelju pomoću html oznaka

[BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) je Python biblioteka za dohvaćanje podataka iz HTML/XML datoteka. BeautifulSoup podržava mnogobrojne [parsere](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser):

| Parser | Typical usage | Advantages | Disadvantages |
| ------ | ------------- | ---------- | ------------- |
| Python’s html.parser | `BeautifulSoup(markup, "html.parser")`| - Batteries included <br> - Decent speed | - Not as fast as lxml, less lenient than html5lib. |
| [**lxml’s HTML parser**](https://lxml.de/) | `BeautifulSoup(markup, "lxml")` | - Very fast | - External C dependency |
| lxml’s XML parser | `BeautifulSoup(markup, "lxml-xml") BeautifulSoup(markup, "xml")` | - Very fast <br> - The only currently supported XML parser | - External C dependency |
| html5lib | `BeautifulSoup(markup, "html5lib")` | - Extremely lenient <br> - Parses pages the same way a web browser does <br> - Creates valid HTML5 | - Very slow <br> - External Python dependency |





##### Osnovno navigiranje pomoću BS4

Dohvatimo za početak stranicu s izvedbenim planovima diplomskog studija: `https://www.inf.uniri.hr/izvedbeni-programi/diplomski-studij-informatike`

In [ ]:
import requests

URL_DIP = "https://www.inf.uniri.hr/izvedbeni-programi/diplomski-studij-informatike"

page_dip = requests.get(URL_DIP)

print(page_dip.text)

Stvorimo novi soup objekt pomoću BeautifulSoup konstruktora:

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page_dip.text, 'html.parser')

print(soup.prettify())

In [ ]:
# Sadržaj pod "title" HTML oznakom
print(soup.title)

# Ime HTML oznake
print(soup.title.name)

# Sadržaj HTML oznake
print(soup.title.string)

# Ime "nadređenog" html elementa
print(soup.title.parent.name)

# Prvi element u stablu s oznakom p - paragraph
print(soup.p)

# span element unutar prvog paragrpah elementa
print(soup.p.span)

# Stil span elementa
print(soup.p.span["style"])

# Prvi element s a - anchor oznakom
print(soup.a)

# Svi elemeti sa span oznakom
print(soup.find_all('span'))

# Prvi element s danim id parametrima
soup.find(id="copyright-year")


> ZADATAK 5.  Pronađite element kojem je klasa "item-294".

In [ ]:
# class je ključna riječ koja je rezervirana za definiranje klase!
print(soup.find(class_="item-294"))

> ZADATAK 6. Pronađite sve elemente kojima je klasa "item-N", pri čemu je N troznamenkasti broj

In [ ]:
import re
print(soup.find_all(class_=re.compile(r"item-[0-9]{3}")))

> ZADATAK 3* 

In [ ]:
linkovi_dip = []
retci = soup.find_all('td')
for r in retci:
    try:
        print(r.find('a')["href"])
        linkovi_dip.append(r.find('a')["href"])
    except:
        pass

#### Preuzimanje sadržaja (PDF)

Do sada smo dohvatili sve URL-ove koji su nam interesantni (`linkovi`, `linkovi_dip`), potrebno je dostupne (.pdf) datoteke pohraniti na lokalno računalo/disk -> Ne želimo isponova dohvaćati podatke kada za to postoji potreba!


> ZADATAK 7. Preuzmite PDF dokumente na lokalno računalo.

In [ ]:
# Spajanje listi
pdf_url = linkovi
pdf_url.extend(linkovi_dip)

# Čišćenje duplikata - gubi se poredak
pdf_url = list(set(pdf_url))

# Zanimaju nas isključivo URL-ovi PDF-a
pdf_url = [p for p in pdf_url if p.endswith(".pdf")]

print(len(pdf_url))

Primjer:

In [ ]:
primjer_url = "https://www.inf.uniri.hr" + pdf_url[10]

with open(f'./DINPovi/{primjer_url.split("/")[-1]}', 'xb') as f:
    f.write(requests.get(primjer_url).content)


In [ ]:
from random import random
from time import sleep

# URL web sjedišta
WEBSITE_URL = "https://www.inf.uniri.hr"


for link in pdf_url:
    sleep(random()) # Nasumični delay od 0 do 1 sekunde
    url = WEBSITE_URL + link
    print(url)
    try:
        with open(f'./DINPovi/{url.split("/")[-1]}', 'xb') as f:
            f.write(requests.get(WEBSITE_URL + link).content)
    except FileExistsError:
        print("Already downloaded")
